# Photo Classifier: Hyperparameter tuning

## Imports

In [1]:
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline
%load_ext autoreload
%autoreload 2
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import logging

In [3]:
from file_io import load_pickle_file
from constants import *
from utils import *
from model import get_pretrained_model_for_transfer_learning
from learners.utils import get_learner
from model_optimizer import ModelOptimizer
from image_transforms import *

In [4]:
if RANDOM_SEED:
    set_random_seeds(RANDOM_SEED)
logging.basicConfig(level=logging.INFO)

## Load model and data

In [5]:
image_data = load_pickle_file(IMAGE_DATA_PATH)

In [6]:
n_classes = len(image_data.labels)
model = get_pretrained_model_for_transfer_learning(n_classes, IS_MULTILABEL_CLASSIFICATION, DROPOUT, MODEL_ARCHITECTURE)
Learner = get_learner(IS_MULTILABEL_CLASSIFICATION)
learner = Learner(model, use_gpu=USE_GPU)

## Hyperparameter tuning

In [7]:
parameter_options = {
    'learning_rate': [0.002*k for k in range(10)],
    'weight_decay': [0.002*k for k in range(10)],
    'epochs': [5],
    'batch_size': [5, 10, 15],
    'image_transforms_training': [TransformsTraining],
    'image_transforms_validation': [TransformsTest],
    'early_stop_option': [False],
}

In [8]:
optimizer = ModelOptimizer(learner, image_data, parameter_options)
results = optimizer.run_random_search(3)

INFO:model_optimizer:1/3
INFO:learners.base_learner:
            Epoch: 1/5
            Validation loss: 0.8120878338813782
INFO:learners.base_learner:
            Epoch: 2/5
            Validation loss: 0.48705437779426575
INFO:learners.base_learner:
            Epoch: 3/5
            Validation loss: 0.6190658807754517
INFO:learners.base_learner:
            Epoch: 4/5
            Validation loss: 0.7238577008247375
INFO:learners.base_learner:
            Epoch: 5/5
            Validation loss: 0.329302579164505
INFO:learners.base_learner:Lowest validation loss: epoch: 5; loss: 0.329302579164505
INFO:model_optimizer:2/3
INFO:learners.base_learner:
            Epoch: 1/5
            Validation loss: 3.9102516174316406
INFO:learners.base_learner:
            Epoch: 2/5
            Validation loss: 0.41550663113594055
INFO:learners.base_learner:
            Epoch: 3/5
            Validation loss: 0.47529852390289307
INFO:learners.base_learner:
            Epoch: 4/5
            Validati

In [9]:
results

,batch_size,early_stop_option,epochs,image_transforms_training,image_transforms_validation,learning_rate,weight_decay,lowest_validation_loss,time
2,15,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.006,0.016,0.203807,9.516833
0,5,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.002,0.000,0.329303,12.798655
1,5,False,5,"Compose(\n RandomResizedCrop(size=(256, 256...","Compose(\n Resize(size=256, interpolation=P...",0.016,0.002,0.415507,10.086663


In [10]:
best_learner = optimizer.get_best_learner()